# Поиск аномалий (система алёртов)

#### Задание

##### 1. Написать систему алертов для нашего приложения. 
##### 2. Система должна с периодичность каждые 15 минут проверять ключевые метрики, такие как активные пользователи в ленте / мессенджере, просмотры, лайки, CTR, количество отправленных сообщений. 
##### 3. Изучить поведение метрик и подберите наиболее подходящий метод для детектирования аномалий. На практике как правило применяются статистические методы. 
* В самом простом случае можно, например, проверять отклонение значения метрики в текущую 15-минутку от значения в такую же 15-минутку день назад. 
* В случае обнаружения аномального значения, в чат должен отправиться алерт - сообщение со следующей информацией: метрика, ее значение, величина отклонения.

##### 4. Автоматизировать запуск с помощью GitLab CI/CD. 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import telegram
import pandahouse
from datetime import date
import io
from read_db.CH import Getch
import sys
import os



def check_anomaly(df, metric, a=4, n=5):
    #Алгорит поиска аномалий данных (межквартильный размах)
    df['q25'] = df[metric].shift(1).rolling(n, center = True, min_periods = 1).quantile(0.25)
    df['q75'] = df[metric].shift(1).rolling(n, center = True, min_periods = 1).quantile(0.75)
    df['iqr'] = df['q75'] - df['q25']
    df['up'] = df['q75'] + a*df['iqr']
    df['low'] = df['q25'] - a*df['iqr']

    #Проверям данные за последнюю 15-ти минутку на наличие аномалий
    #путём сравнения нашего значением с нижней и верхней границы гайдлайна               
    if df[metric].iloc[-1] < df['low'].iloc[-1] or df[metric].iloc[-1] > df['up'].iloc[-1]:
        is_alert = 1
    else:
        is_alert = 0
                   
    return is_alert, df

def run_alerts(chat=None):    
    #Сама система алертов
    chat_id = chat or 726885196 
    bot = telegram.Bot(token='5388356743:AAFRvvBIu_oe-DprC7UJNiTajKpFf72QSnE')
    
    #Берём данные по ленте новостей
    data_feed = Getch('''SELECT toStartOfFifteenMinutes(time) as ts
                            , toDate(time) as date
                            , formatDateTime(ts, '%R') as hm
                            , uniqExact(user_id) as user_feed
                            , countIf(user_id, action = 'like') as like
                            , countIf(user_id, action = 'view') as view
                            , round(like / view, 2) as CTR
                            FROM simulator_20220520.feed_actions
                            WHERE time >= today()-1 AND time < toStartOfFifteenMinutes(now())
                            GROUP BY ts, date, hm
                            ORDER BY ts''').df
    
    #Берём данные по мессенджеру
    data_message = Getch('''SELECT toStartOfFifteenMinutes(time) as ts
                            , toDate(time) as date
                            , formatDateTime(ts, '%R') as hm
                            , uniqExact(user_id) as user_message
                            , uniqExact(reciever_id) as reciever_message
                            , 'message' as action
                            , countIf(user_id, action = 'message') as message
                            FROM simulator_20220520.message_actions
                            WHERE time >= today()-1 AND time < toStartOfFifteenMinutes(now())
                            GROUP BY ts, date, hm
                            ORDER BY ts''').df
    
    metrics_list_feed = ['user_feed', 'like', 'view', 'CTR']
    metrics_list_message = ['user_message', 'message', 'reciever_message']

    #Делаем цикл для проверки каждой метрики из списка
    for metric in metrics_list_feed:
        print(metric)
        df = data_feed[['ts', 'date', 'hm', metric]].copy()
        is_alert, df = check_anomaly(df, metric)
        
    #Если система обнаруживает отклонение от наших гайдлайнов, то бот отправляет сообщение со всей информацией и строит график, на котором видна аномалия
        if is_alert == 1:
            msg = '''Метрика {metric}:\n текущее значение {current_val:.2f}\nотклонение от предыдущего значения {last_val_diff:.2%}\nhttps://superset.lab.karpov.courses/superset/dashboard/1024/'''.format(metric = metric,
                                                                                                                                       current_val = df[metric].iloc[-1],
                                                                                                                                       last_val_diff =abs(1 - (df[metric].iloc[-1] / df[metric].iloc[-2])))

            sns.set(rc = {'figure.figsize' : (16, 10)})
            plt.tight_layout()

            ax = sns.lineplot(x = df['ts'] , y = df[metric], label = 'metric')
            ax = sns.lineplot(x = df['ts'] , y = df['up'], label = 'up')
            ax = sns.lineplot(x = df['ts'] , y = df['low'], label = 'low')

            for ind, label in enumerate(ax.get_xticklabels()):
                if ind % 1 == 0:
                    label.set_visible(True)
                else:
                    label.set_visible(False)

            ax.set(xlabel = 'time')
            ax.set(ylabel = metric)

            ax.set_title(metric)
            ax.set(ylim = (0, None))

            plot_object = io.BytesIO()
            ax.figure.savefig(plot_object)
            plot_object.seek(0)
            plot_object.name = '{0}.png'.format(metric)
            plt.close()

            bot.sendMessage(chat_id=chat_id, text=msg)
            bot.sendPhoto(chat_id=chat_id, photo=plot_object)
            
    #Делаем цикл для проверки каждой метрики из списка        
    for metric in metrics_list_message:
        print(metric)
        df_m = data_message[['ts', 'date', 'hm', metric]].copy()
        is_alert, df_m = check_anomaly(df_m, metric)
        
    #Если система обнаруживает отклонение от наших гайдлайнов, то бот отправляет сообщение со всей информацией и строит график, на котором видна аномалия
        if is_alert == 1:
            msg = '''Метрика {metric}:\n текущее значение {current_val:.2f}\nотклонение от предыдущего значения {last_val_diff:.2%}\nhttps://superset.lab.karpov.courses/superset/dashboard/1024/'''.format(metric = metric,
                                                                                                                                       current_val = df_m[metric].iloc[-1],
                                                                                                                                       last_val_diff =abs(1 - (df_m[metric].iloc[-1] / df_m[metric].iloc[-2])))

            sns.set(rc = {'figure.figsize' : (16, 10)})
            plt.tight_layout()

            ax = sns.lineplot(x = df_m['ts'] , y = df_m[metric], label = 'metric')
            ax = sns.lineplot(x = df_m['ts'] , y = df_m['up'], label = 'up')
            ax = sns.lineplot(x = df_m['ts'] , y = df_m['low'], label = 'low')

            for ind, label in enumerate(ax.get_xticklabels()):
                if ind % 1 == 0:
                    label.set_visible(True)
                else:
                    label.set_visible(False)

            ax.set(xlabel = 'time')
            ax.set(ylabel = metric)

            ax.set_title(metric)
            ax.set(ylim = (0, None))

            plot_object = io.BytesIO()
            ax.figure.savefig(plot_object)
            plot_object.seek(0)
            plot_object.name = '{0}.png'.format(metric)
            plt.close()

            bot.sendMessage(chat_id=chat_id, text=msg)
            bot.sendPhoto(chat_id=chat_id, photo=plot_object)

    

try:
    run_alerts(-652068442)
except Exception as e:
    print(e)


user_feed
like
view
CTR
user_message
message
reciever_message
